In [9]:
import tensorflow as tf
import pickle
import numpy as np
import timeit
import load_cifar as data
import sys

import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [10]:
tf.reset_default_graph()

<h1>Define Hyper-perparmeter</h1>

In [11]:
epochs = 10;
batch_size = 128
lr = 0.001

# Probability for how many units are kep in each layer (dropout technique)
dr = 0.7

<h1>Define Placeholder</h1>

In [12]:
tf.reset_default_graph()

# Define placeholders
x = tf.placeholder(tf.float32, shape=(None, 32, 32, 3), name='input')
y = tf.placeholder(tf.float32, shape=(None, 10), name='output')
d = tf.placeholder(tf.float32, name="droput_rate")

<h1>Define Neural Network Architecture</h1>

In [13]:
def neural_net(x, dropout_rate):

    ### Define convolutional layers ###

    # 64 filters, 3x3
    filt1 = tf.Variable(tf.truncated_normal(shape=[3,3,3,64], mean=0, stddev=0.08))
    filt2 = tf.Variable(tf.truncated_normal(shape=[3, 3, 64, 128], mean=0, stddev=0.08))
    filt3 = tf.Variable(tf.truncated_normal(shape=[5, 5, 128, 256], mean=0, stddev=0.08))
    
    # Convolution/Pooling layers
    c1 = tf.nn.conv2d(x, filt1, strides=[1,1,1,1], padding='SAME')
    c1 = tf.nn.relu(c1)
    p1 = tf.nn.max_pool(c1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    cn1 = tf.layers.batch_normalization(p1)
    
    c2 = tf.nn.conv2d(cn1, filt2, strides=[1,1,1,1], padding='SAME')
    c2 = tf.nn.relu(c2)
    p2 = tf.nn.max_pool(c2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')    
    cn2 = tf.layers.batch_normalization(p2)
    
    #c3 = tf.nn.conv2d(c2, filt3, strides=[1,1,1,1], padding='SAME')
    #c3 = tf.nn.relu(c3)
    #p3 = tf.nn.max_pool(c3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')  
    #cn3 = tf.layers.batch_normalization(p3)


    ### Define fully connected layers ###

    # Flatten the output
    flat = tf.contrib.layers.flatten(cn2)

    # Fully connected layer (128)
    f1 = tf.contrib.layers.fully_connected(inputs=flat, num_outputs=128, activation_fn=tf.nn.relu)
    f1 = tf.nn.dropout(f1, dr)
    f1 = tf.layers.batch_normalization(f1)
    
    out = tf.contrib.layers.fully_connected(inputs=f1, num_outputs=10, activation_fn=tf.nn.relu)


    return out




<h1>Define Cost and Optimization</h1>

In [14]:
logits = neural_net(x, dr)

# Define cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

# Define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(cost)

# Define accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

<h1>Training and testing</h1>
<h2>1.Print out validation accuracy after each training poch</h2>
<h2>2.Print out training time you spend on each epoch</h2>
<h2>3.Print out testing accuracy in the end</h2>

In [15]:
def train_neural_net(session, optimizer, dr, features, labels):
    session.run(optimizer,
                feed_dict={
                    x: features,
                    y: labels,
                    d: dr
                })
    
def get_stats(session, cost, accuracy, b_feat, b_labels, v_feat, v_labels):
    loss = sess.run(cost,
                   feed_dict={
                       x: b_feat,
                       y: b_labels, 
                       d: 1.
                   })
    
    acc = sess.run(accuracy, 
                  feed_dict={
                      x: v_feat,
                      y: v_labels,
                      d: 1.
                  })
    
    print('Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(loss, acc))

In [16]:
#Initialize the variables
init = tf.global_variables_initializer()

v_feat, v_labels = pickle.load(open('preprocessed_validation.p', mode='rb'))

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        
        n_batches = 5
        for batch_i in range(1, n_batches+1):
            for batch_features, batch_labels in data.load_preprocessed_training_batch(batch_i, batch_size):
                train_neural_net(sess, optimizer, dr, batch_features, batch_labels)

            print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, batch_i), end='')
            get_stats(sess, cost, accuracy, batch_features, batch_labels, v_feat, v_labels)
                

Epoch  1, CIFAR-10 Batch 1:  Loss:     2.2067 Validation Accuracy: 0.211200
Epoch  1, CIFAR-10 Batch 2:  Loss:     1.7165 Validation Accuracy: 0.362200
Epoch  1, CIFAR-10 Batch 3:  Loss:     1.3938 Validation Accuracy: 0.435000
Epoch  1, CIFAR-10 Batch 4:  Loss:     1.3542 Validation Accuracy: 0.486000
Epoch  1, CIFAR-10 Batch 5:  Loss:     1.3098 Validation Accuracy: 0.505200
Epoch  2, CIFAR-10 Batch 1:  Loss:     1.4718 Validation Accuracy: 0.516000
Epoch  2, CIFAR-10 Batch 2:  Loss:     1.1552 Validation Accuracy: 0.573200
Epoch  2, CIFAR-10 Batch 3:  Loss:     0.8672 Validation Accuracy: 0.570200
Epoch  2, CIFAR-10 Batch 4:  Loss:     0.8425 Validation Accuracy: 0.594200
Epoch  2, CIFAR-10 Batch 5:  Loss:     1.0920 Validation Accuracy: 0.612200
Epoch  3, CIFAR-10 Batch 1:  Loss:     1.1369 Validation Accuracy: 0.596400
Epoch  3, CIFAR-10 Batch 2:  Loss:     0.8816 Validation Accuracy: 0.631400
Epoch  3, CIFAR-10 Batch 3:  Loss:     0.6874 Validation Accuracy: 0.640200
Epoch  3, CI

<h3>Discussion</h3>

<p style="font-size:15px">
I acheived the above results, by slowly adding complextity to the neural network architecture to exceed 50% accuracy. I started off with just a fully connected network (10 nodes) and the accuracy was very low. Then I added a convolutional and pooling layer before the fully connected layer. This acheived an accuracy of around 30%. I also lowered the learning rate from 0.1 to 0.001 </p>
<p style="font-size:15px">
Next, I tried adding more convolution + pooling layers, but the accuracy got worse. I went back to just two convolution + pooling layers and then I added a fully connected layer with 128 nodes before my final fully connected layer of 10 nodes and I was able to achieve 68% accuracy within 10 epochs. In fact, 50% accuracy was reached in the first epoch. </p>